# Segmenting and Clustering Neighborhoods in Toronto -Part 3

##  This notebook includes part 1 and part 2 code to run independantly. The main objective is to explore the Toronto neighbourhood simiar to the lab demo. 

## Note: The assignment instruction isn't clear about what exactly need to be analyzed, therefore the lab code is used here to generate similar analysis result. 

## Please notify if there is more clear instruction and this notebook can be improved!!!
## Thanks again.

###  Due to my windows environment, some libraries were installed locally using Anaconda.
 

In [37]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import geocoder # import geocoder

#!conda install -c conda-forge geopy --yes 

# uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#!conda install -c conda-forge geocoder 
import folium # map rendering library

import requests
import geopy.geocoders

from lxml import html
print('Libraries imported.')


Libraries imported.


In [38]:
df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#results=requests.get(url)
df=pd.read_html(url)
tb = pd.DataFrame(columns=['Postal Code', 'Borough', 'Neighborhood'])
tb=pd.read_html(url,header=0)[0]
#tb.to_csv('./output.csv',mode='a', encoding='utf-8', header=0, index=0)
#df.head(6)
tb.head(10)



,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [39]:
tb_clean=tb[tb.Borough !='Not assigned']


pd.set_option('display.width',2000)


print(tb_clean)
#tb_clean.to_csv('./tb_cleanoutput.csv',mode='a', encoding='utf-8', header=0, index=0)



    Postcode           Borough                                      Neighbourhood
2        M3A        North York                                          Parkwoods
3        M4A        North York                                   Victoria Village
4        M5A  Downtown Toronto                                       Harbourfront
5        M5A  Downtown Toronto                                        Regent Park
6        M6A        North York                                   Lawrence Heights
7        M6A        North York                                     Lawrence Manor
8        M7A      Queen's Park                                       Not assigned
10       M9A         Etobicoke                                   Islington Avenue
11       M1B       Scarborough                                              Rouge
12       M1B       Scarborough                                            Malvern
14       M3B        North York                                    Don Mills North
15       M4B    

In [40]:
tb_clean_merge=tb_clean.groupby(['Postcode','Borough']).apply(lambda x:[','.join(x['Neighbourhood'])])
#tb_clean_merge=tb_clean['Postcode'].groupby('Borough').apply(lambda x:[','.join(x['Neighbourhood'])])
#tb_clean_merge.head(10)

#
tb_clean_merge=tb_clean_merge.to_frame().reset_index()
#tb_clean_merge.rename(columns={"Postcode","Borough","Neighbourbood"})
tb_clean_merge.columns=['Postal Code','Borough','Neighbourhood']



#type(tb_clean_merge)
print(tb_clean_merge)

tb_clean_merge.shape

    Postal Code           Borough                                      Neighbourhood
0           M1B       Scarborough                                    [Rouge,Malvern]
1           M1C       Scarborough             [Highland Creek,Rouge Hill,Port Union]
2           M1E       Scarborough                  [Guildwood,Morningside,West Hill]
3           M1G       Scarborough                                           [Woburn]
4           M1H       Scarborough                                        [Cedarbrae]
5           M1J       Scarborough                              [Scarborough Village]
6           M1K       Scarborough        [East Birchmount Park,Ionview,Kennedy Park]
7           M1L       Scarborough                    [Clairlea,Golden Mile,Oakridge]
8           M1M       Scarborough    [Cliffcrest,Cliffside,Scarborough Village West]
9           M1N       Scarborough                       [Birch Cliff,Cliffside West]
10          M1P       Scarborough  [Dorset Park,Scarborough Town 

(103, 3)

In [41]:
#pip install geocoder

!curl -s -L -o Toronto.csv https://cocl.us/Geospatial_data
print('Toronto GEO Data donloaded!')

toronto_data = pd.read_csv('./Toronto.csv') 
#data = pd.read_csv("filename.csv") 

#print(toronto_data)





Toronto GEO Data donloaded!


In [42]:
toronto_data.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [43]:


#Pre-flight checking on tb_clean_merge
tb_clean_merge.to_csv('./tb_clean_merge.csv',mode='a', encoding='utf-8', header=0, index=0)
#Pre-flight checking on toronto_data
toronto_data.to_csv('./toronto_data.csv',mode='a', encoding='utf-8', header=0, index=0)




In [44]:
df_final= pd.merge(tb_clean_merge, toronto_data, how='left', on='Postal Code')

In [45]:
df_final.shape
df_final.head(50)
df_final.to_csv('./df_final_data.csv',mode='a', encoding='utf-8', header=0, index=0)

In [46]:
df_final.dtypes

Postal Code       object
Borough           object
Neighbourhood     object
Latitude         float64
Longitude        float64
dtype: object

##  ====>>> Part 3 code starts here <<<====

### Below code first to generate Toronto map, using folium map function. 
### Markers are added for the postal code , based on the dataframe generated from Part 1 and Part 2 code. 


In [47]:
address = 'Toronto,ON'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighbourhood']):
    label = '{}, {}'.format(df_final, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


### The client_id and secret should be recycled for security purpose. 

In [48]:
CLIENT_ID = 'FMEFDACOSZG44GGIQHFEOAH4FKF0CE4202IS13QUEC50WVD2' # your Foursquare ID
CLIENT_SECRET = 'LSYA5RGOBKMKELJGPQJBT4KWV3HYFFZL5KMYIUGTXPR4IJWY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FMEFDACOSZG44GGIQHFEOAH4FKF0CE4202IS13QUEC50WVD2
CLIENT_SECRET:LSYA5RGOBKMKELJGPQJBT4KWV3HYFFZL5KMYIUGTXPR4IJWY


In [49]:
df_final.loc[0, 'Neighbourhood']

['Rouge,Malvern']

In [50]:
neighborhood_latitude = df_final.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_final.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_final.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of ['Rouge,Malvern'] are 43.806686299999996, -79.19435340000001.


### Try to find the Italian restraunt within 500 meters radius of given coordinates.

## Again wish I can get a better notebook but really can't understand what exactly the instruction is asking about. 


In [51]:



search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

venues_query_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
venues_query_url

results = requests.get(venues_query_url).json()
results


venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()


Italian .... OK!


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,51bf3866498e55ee55df8db0,NaN,CA,Toronto,Canada,NaN,127,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65499143746528...",43.654991,-79.387897,NaN,ON,The Fresh Italian,v-1566791920,NaN
1,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...",False,4bfc0289c3ba9521c00f9653,136 Beverley St,CA,Toronto,Canada,Dundas Street,555,"[136 Beverley St (Dundas Street), Toronto ON, ...","[{'label': 'display', 'lat': 43.65402694219784...",43.654027,-79.394104,NaN,ON,Italian Consulate Toronto,v-1566791920,NaN
2,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,526fe29411d2aeb3803013b0,"109 McCaul Street, Unit #42",CA,Toronto,Canada,Dundas Street West,288,"[109 McCaul Street, Unit #42 (Dundas Street We...","[{'label': 'display', 'lat': 43.653889, 'lng':...",43.653889,-79.390785,M5T 3K5,ON,The Fresh Italian Eatery,v-1566791920,NaN
3,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,573df789498e03dd8e54b166,595 Bay St,CA,Toronto,Canada,Dundas St,405,"[595 Bay St (Dundas St), Toronto ON M5G 2C2, C...","[{'label': 'display', 'lat': 43.65616, 'lng': ...",43.656160,-79.383190,M5G 2C2,ON,Mustachio Italian Eatery,v-1566791920,NaN
4,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,4f88cf84e4b002b90ab3b9b9,NaN,CA,NaN,Canada,NaN,434,[Canada],"[{'label': 'display', 'lat': 43.65053979517576...",43.650540,-79.384603,NaN,NaN,LA's Italian + Bar,v-1566791920,NaN


In [52]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Fresh Italian,Italian Restaurant,NaN,CA,Toronto,Canada,NaN,127,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65499143746528...",43.654991,-79.387897,NaN,ON,51bf3866498e55ee55df8db0
1,Italian Consulate Toronto,Embassy / Consulate,136 Beverley St,CA,Toronto,Canada,Dundas Street,555,"[136 Beverley St (Dundas Street), Toronto ON, ...","[{'label': 'display', 'lat': 43.65402694219784...",43.654027,-79.394104,NaN,ON,4bfc0289c3ba9521c00f9653
2,The Fresh Italian Eatery,Italian Restaurant,"109 McCaul Street, Unit #42",CA,Toronto,Canada,Dundas Street West,288,"[109 McCaul Street, Unit #42 (Dundas Street We...","[{'label': 'display', 'lat': 43.653889, 'lng':...",43.653889,-79.390785,M5T 3K5,ON,526fe29411d2aeb3803013b0
3,Mustachio Italian Eatery,Italian Restaurant,595 Bay St,CA,Toronto,Canada,Dundas St,405,"[595 Bay St (Dundas St), Toronto ON M5G 2C2, C...","[{'label': 'display', 'lat': 43.65616, 'lng': ...",43.656160,-79.383190,M5G 2C2,ON,573df789498e03dd8e54b166
4,LA's Italian + Bar,Italian Restaurant,NaN,CA,NaN,Canada,NaN,434,[Canada],"[{'label': 'display', 'lat': 43.65053979517576...",43.650540,-79.384603,NaN,NaN,4f88cf84e4b002b90ab3b9b9
5,Classic italian style pizza food truck,Food Truck,CNE Midway,CA,Toronto,Canada,NaN,530,"[CNE Midway, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.652144, 'lng':...",43.652144,-79.381118,NaN,ON,4c787c9181bca0936180fa14
6,john's italian cafe,Italian Restaurant,27 Baldwin Street,CA,Toronto,Canada,NaN,546,"[27 Baldwin Street, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65612672798775...",43.656127,-79.393301,NaN,ON,53daae5b498e9c9597c19b23
7,Little Anthony's,Italian Restaurant,121 Richmond St. W,CA,Toronto,Canada,at York St.,462,"[121 Richmond St. W (at York St.), Toronto ON ...","[{'label': 'display', 'lat': 43.65029624519052...",43.650296,-79.384513,M5H 2K1,ON,4b846dd4f964a520dc3431e3
8,Sbarro,Pizza Place,220 Yonge St.,CA,Toronto,Canada,in Toronto Eaton Centre,526,"[220 Yonge St. (in Toronto Eaton Centre), Toro...","[{'label': 'display', 'lat': 43.655518, 'lng':...",43.655518,-79.381030,M5B 2H1,ON,4b4a2d09f964a520687d26e3


In [53]:
dataframe_filtered.name
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map